In [65]:
study = "1.2.392.200036.9116.2.5.1.48.1216060901.1336345628.42"
series = "1.2.392.200036.9116.2.1216060901.1336346550.3.1004100001.7"
instance = "1.2.392.200036.9116.2.1216060901.1336346841.684518.7.1277"
query_vna(user='wangkcc4', pw='', filename="test.dcm", study=study, series=series, instance=instance)

In [62]:
import requests

def query_vna(user, pw, filename, study=None, series=None, instance=None, region='test', metadata=False):

    if region == 'test':
        host = 'vnatest1vt'
        port = '8083'
    elif region == 'prod':
        host = 'TBD'
        port = 'TBD'
    else:
        raise ValueError("Unsupported region")


    if metadata:
        url = ''.join(['http://', host, ':', port,
                       "/AcuoREST/dicomrs/retrieve/studies/",
                        study])

        if series is not None:
            url += "/series/" + series
            if instance is not None:
                url += "/instances/" + instance

        url += "/metadata"

    else:
        url = ''.join(['http://', host, ':', port,
                       "/AcuoREST/wadoget?requestType=WADO&contentType=application/dicom&studyUID=",
                        study])

        if series is not None:
            url += "&seriesUID=" + series
            if instance is not None:
                url += "&objectUID=" + instance

    r = requests.get(url, auth=(user, pw)) #, headers=headers

    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
            
    if r.status_code != 200:
        raise ValueError("Invalid request (response code %d)" % r.status_code)

In [60]:
r.headers

{'Transfer-Encoding': 'chunked', 'Cache-Control': 'no-cache', 'Access-Control-Allow-Origin': '*', 'Date': 'Fri, 22 Sep 2017 20:53:33 GMT', 'Content-Type': 'multipart/related; boundary=55EDEE38B0E7054DB8704E8FC21B7D4F;start="0@8F590402E9196D4EBBD9D9901038EB85";type="application/dicom+xml"', 'Access-Control-Allow-Methods': 'GET, OPTIONS', 'Server': 'Microsoft-HTTPAPI/2.0', 'Access-Control-Allow-Headers': 'Origin, Content-Type, Accept, Authorization'}

In [61]:
r.status_code

200